In [ ]:
pip install tensorflow==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully unin

In [ ]:
import tensorflow as tf; print(tf.__version__)

2.15.0


In [ ]:
pip install coremltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
import coremltools as ct
import tensorflow as tf
from tensorflow import keras

def convert_h5_to_mlmodel(h5_path, mlmodel_path, model_name="MyModel"):
    """
    Convert H5 model to Core ML format

    Args:
        h5_path: Path to the .h5 file
        mlmodel_path: Output path for the .mlmodel file
        model_name: Name for the Core ML model
    """

    # Load the H5 model
    print(f"Loading model from {h5_path}...")
    model = keras.models.load_model(h5_path)

    # Print model summary
    print("Model architecture:")
    model.summary()

    # Convert to Core ML
    print("Converting to Core ML format...")

    # For TensorFlow/Keras models
    coreml_model = ct.convert(
        model,
        source="tensorflow",
        convert_to="mlprogram",  # Use "neuralnetwork" for older iOS versions
        compute_units=ct.ComputeUnit.ALL,  # CPU_AND_GPU, CPU_ONLY, or ALL
        minimum_deployment_target=ct.target.iOS15  # Adjust as needed
    )

    # Set model metadata
    coreml_model.short_description = f"Converted {model_name}"
    coreml_model.author = "Your Name"
    coreml_model.license = "MIT"
    coreml_model.version = "1.0"

    # Save the model
    print(f"Saving Core ML model to {mlmodel_path}...")
    coreml_model.export(mlmodel_path)

    print("Conversion completed successfully!")
    return coreml_model

# Example usage
if __name__ == "__main__":
    # Replace with your actual file paths
    h5_file = "./model_20250707-231404.h5"
    output_file = "./class_labels_20250707-231405.json"

    # Convert the model
    try:
        coreml_model = convert_h5_to_mlmodel(h5_file, output_file, "MyConvertedModel")

        # Print input/output specifications
        print("\nModel Input Specifications:")
        print(coreml_model.input_description)
        print("\nModel Output Specifications:")
        print(coreml_model.output_description)

    except Exception as e:
        print(f"Error during conversion: {e}")
        print("Make sure your H5 file is valid and coremltools is installed")

Loading model from ./model_20250707-231404.h5...
Error during conversion: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 256, 256, 1], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']
Make sure your H5 file is valid and coremltools is installed


In [ ]:
import tensorflow as tf

# Define your Sequential model, using 'input_shape' for the first layer
# IMPORTANT: Replace these layers with your actual model's layers and parameters
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)), # EXAMPLE: use your actual input shape
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax') # EXAMPLE: use your actual output layer
])

# If your model was compiled with specific settings, you can compile it here.
# This isn't strictly necessary for loading weights, but good practice.
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the summary to ensure the architecture is correct
model.summary()

# Load the weights from your original .h5 file
# This assumes your .h5 file contains only weights, OR if load_model fails
# due to architecture issues, you load weights separately.
try:
    model.load_weights('model_20250707-231404.h5')
    print("Successfully loaded weights onto the newly defined model.")
except Exception as e:
    print(f"Error loading weights: {e}")
    print("Double-check your model architecture definition and the .h5 file path.")
    # If model.load_weights also fails, it implies a deeper mismatch or corruption.
    exit() # Stop if weights can't be loaded

# Now, save the model in the current TensorFlow version's format
new_model_path = 'model_ready.h5'
model.save(new_model_path)
print(f"Model successfully saved to {new_model_path}")

# You can now use 'your_updated_model_fixed.h5' for your conversions

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 128)               3936384   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3938570 (15.02 MB)
Trainable params: 3938570 (15.02 MB)
Non-trainable params: 0 (0.00 Byte)
________________

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
